In [1]:
# einige Paketten wurden für Visualisierung gebraucht
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
#import umap.umap_ as umap
import time
#import plotly.express as px
#from sklearn import cluster
#from sklearn import metrics

from src.prepare_dataset import TextDataLoader
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
min_df= 30
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=min_df, stopwords_remove_from_voca=True)

# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False
word2id, id2word, train_set, test_set, val_set = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

loading texts: ...
train-size after loading: 11314
test-size after loading: 7532
finished load!
start: preprocessing: ...
finised: preprocessing!
vocab-size in df: 8496
validation-size ist: 0.01
start creating vocabulary ...
length of the vocabulary: 8496
sample ten words of the vocabulary: ['determines', 'cv', 'chung', 'dealing', 'tall', 'succeeded', 'sufficiently', 'oracle', 'distribute', 'compatible']
length word2id list: 8496
length id2word list: 8496
finished: creating vocabulary
train-size-after-all: 11214
test-size-after-all: 7532
validation-size-after-all: 100
test-size-after-all: 11214
test-indices-length: 11214
test-size-after-all: 100
test-indices-length: 100
test-size-after-all: 7532
test-indices-length: 7532
length train-documents-indices : 1150368
length of the vocabulary: 8496


start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow input!

start: creating bow representation...
finised creating bow in

,word,id
0,determines,0
1,cv,1
2,chung,2
3,dealing,3
4,tall,4
...,...,...
95,existance,95
96,viewpoint,96
97,documented,97
98,issue,98


In [2]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train_set["tokens"])}')
print(f'Size of val set: {len(val_set["tokens"])}')
print(f'Size of test set: {len(test_set["test"]["tokens"])}')

# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

Size of the vocabulary after prprocessing ist: 8496
Size of train set: 11214
Size of val set: 100
Size of test set: 7532


,text-after-preprocessing
0,biochem nwu jackson swimming pool defense nntp...
1,apollo hp red herring police state usa nntp po...
2,hades coos dartmouth brian hughes installing r...
3,jaeger buphy bu gregg jaeger rushdie boston un...
4,king eng umd doug boom computer design lab mar...
...,...
95,physics ca campbell pc windows os unix reply p...
96,ncr jim sharp parts information distribution w...
97,sera zuma serdar argic nazi germany armenians ...
98,chips astro temple charlie mathew bible resear...


In [3]:
from scipy.spatial import distance

In [4]:
def argsort(x, topn=None, reverse=False):
    x = np.asarray(x)  # unify code path for when `x` is not a np array (list, tuple...)
    if topn is None:
        topn = x.size
    if topn <= 0:
        return []
    if reverse:
        x = -x
    if topn >= x.size or not hasattr(np, 'argpartition'):
        return np.argsort(x)[:topn]
    # np >= 1.8 has a fast partial argsort, use that!
    most_extreme = np.argpartition(x, topn)[:topn]
    return most_extreme.take(np.argsort(x.take(most_extreme)))  # resort topn into order

In [5]:
# training file for embedding
# options: cbow, skipgram and bert-embedding
# returns: word-embedding for each word in the vocabulary
# inputs: train-documents in words and the vocabulary (?)

import gensim
import pickle
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from numpy import dot, argmax, indices

def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return norm

def unit_vec(vector):
      veclen = np.sqrt(np.sum(vector ** 2))
      #other_veclen = normalize(vector)
      #print(f'veclen: {veclen}')
      #print(f'other-veclen: {other_veclen}')
      return vector/veclen
def get_consine_similarity_2(vector1, vector2):
      #vector unit length so that just use dot product
      #print(np.array(vector1).shape)
      vector1 = unit_vec(np.array(vector1))
      vector2 = unit_vec(np.array(vector2))
      #print(f'veclen after normalization: {normalize(vector1)}')
      return dot(vector1, vector2)

def get_consine_similarity(vector1, vector2):
      #print("using distance.cosine")
      #vector unit length so that just use dot product
      #print(np.array(vector1).shape)
      vector1 = unit_vec(np.array(vector1))
      vector2 = unit_vec(np.array(vector2))
      #print(f'veclen after normalization: {normalize(vector1)}')
      return 1-distance.cosine(vector1, vector2) #1-dot()/norm

def get_similar_vectors_to_given_vector(topn, vocab, give_vector, all_vectors):
      print(vocab[2539])
      dists = []
      for vector2 in all_vectors:
            dists.append(get_consine_similarity_2(give_vector, vector2))
      #dists = np.array(dists)
      print(sorted(dists, reverse=True)[:10])
      top_dists = sorted(dists, reverse=True)[1:topn+1]
      print(top_dists)
      top_indices = [dists.index(d) for d in top_dists]
      #[i for i in range(0, sorted(dists, reverse_))][1:topn+1]
      #list(argsort(dists, reverse=False)[1:topn+1]) #do not use dist = 0 of same vectors
      print(top_indices)
      top_words = {} #np.array(vocab)[indices]
      for idx in top_indices:
            top_words[vocab[idx]] = dists[idx]
      return top_words

def compare_word2vec_methods_and_bert_embeddings(word, word2vec_embeddings, bert_embeddings):
      return {'word2vec': [], 'bert': []}

def read_prefitted_embedding(model_name, vocab, save_path):
      try:
            save_path = Path.joinpath(save_path, f'{model_name}_vocab_embedding.txt')
      except:
            save_path = Path.joinpath(save_path, f'vocab_embedding.txt')

      with open(save_path) as f:
            lines = f.readlines()
      embedding_data = {}
      for t in lines:
            w = t.split("\t")[0]
            v = [float(e) for e in t.split("\t")[1].split(" ")]
            # check again only word in the etm-vocabulary
            if w in vocab:
                  embedding_data[w] = v
      
      # sort embedding_data again by the ordner of the vocabulary from bow
      words_embeddings = np.array(list(embedding_data.values()))
      words = np.array(list(embedding_data.keys()))
      if words == np.array(vocab):
            indices = [vocab.index(words[i]) for i in range(0,len(words))]
            words_in_vocab = [vocab[i] for i in range(0,len(words))]
            words_embeddings = [e for _, e in sorted(zip(indices, words_embeddings))]
            return words_in_vocab, words_embeddings #list(embedding_data.values())
      else:
            print("something wrong at the embedding.py/read_prefitted_embeddings")
            print("use for testing bert")
            return words, words_embeddings #list(embedding_data.values())


In [6]:
class WordEmbeddingCreator:
      def __init__(self, model_name="cbow", documents = None, save_path = ""):
            """
            Input: documents in List of words, train-settings
            Output: word-embedding for the vocabulary

            Args:
                model_name (str, optional): _description_. Defaults to "cbow".
                documents (_type_, optional): _description_. Defaults to None.
                vocab (_type_, optional): _description_. Defaults to None.
                save_path (str, optional): _description_. Defaults to "".
            """
            self.model_name = model_name
            self.save_path = save_path
            self.documents = documents
            self.model = None
            self.all_embeddings = []
            
      def train(self, min_count = 0, embedding_size = 300):
            if self.model_name=="cbow":
                  print("word-embedding train begins")
                  self.model = gensim.models.Word2Vec(self.documents, 
                                                      seed = 42,
                                                      min_count=min_count, 
                                                      sg=0, 
                                                      window=5,
                                                      size=embedding_size,
                                                      iter=5)
            elif self.model_name=="skipgram":
                  print("train begin:word-embedding with skipgram")
                  self.model = gensim.models.Word2Vec(self.documents, 
                                                      seed = 42,
                                                      min_count=min_count, 
                                                      sg=1, 
                                                      window=5,
                                                      size=embedding_size,
                                                      negative=1,
                                                      iter=5)
            else:
                  print("word-embedding with BERT")
                  print("!!!! please run src/bert_main.py to get prepared_data/bert_vocab_embeddings.txt")
            print("word-embedding train finished")
            # todo: save the trained-model
            #return self.model
          
      def create_and_save_vocab_embedding(self, train_vocab = None, embedding_path = None):
            """_summary_

            Args:
                train_vocab (_type_): vocabulary from the prepare_dataset.py
                embedding_path (_type_): path to save the trained-embedding

            Returns:
                _type_: _description_
            """
            model_vocab = []
            if self.model_name=="bert":
                  model_vocab = [] #bert in other processing, ignore here
            else:
                  model_vocab = list(self.model.wv.vocab)
            print(f'length of vocabulary from word-embedding model {len(model_vocab)}')
            print(f'length of the vocabulary of prepraring-dataset-vocabulary: {len(train_vocab)}')
            del self.documents
            
            f = open(Path.joinpath(embedding_path, f'{self.model_name}_vocab_embedding.txt'), 'w') #add to prepared_data
            # sort words in embedding matrix by the ordner from vocabulary
            for v in tqdm(train_vocab): # sort the list embeddings by words in vocabulary
                if v in model_vocab:
                    vec = list(self.model.wv.__getitem__(v))
                    self.all_embeddings.append(vec)
                    f.write(v + '\t')
                    vec_str = ['%.9f' % val for val in vec]
                    vec_str = " ".join(vec_str)
                    f.write(vec_str + '\n')
            f.close()
            self.model.save(str(Path.joinpath(embedding_path, f'{self.model_name}_word2vec.model')))
            return True
      
      def other_save_embeddings(self, train_vocab):
            all_embeddings = []
            model_vocab = list(self.model.wv.vocab)
            for v in tqdm(train_vocab): # sort the list embeddings by words in vocabulary
                 if v in model_vocab:
                       vec = list(self.model.wv.__getitem__(v))
                       all_embeddings.append(vec)
            np.save(f'{self.model_name}_other_embedding.npy', all_embeddings)
            return all_embeddings

      def find_most_similar_words(self, n_neighbor=20, word = None):
            if word!=None:
                  return self.model.wv.most_similar(word, topn=n_neighbor)
            else:
                  print(f'give a word to get the {n_neighbor} neighbor words')

      def find_similar_words_self_implemented(self, topn, train_vocab, word):
            top_words = {}
            model_vocab = list(self.model.wv.vocab)
            #all_embeddings = self.other_save_embeddings(train_vocab)
            if word in train_vocab:
                  if word in model_vocab:
                        considered_vector = list(self.model.wv.__getitem__(word))
                        top_words = get_similar_vectors_to_given_vector(topn, train_vocab, considered_vector, self.all_embeddings)
            return top_words

      def cluster_words(self, embedding_save_path = None, fig_path = None, n_components=3, text = False):
            import umap.umap_ as umap
            import time
            import plotly.express as px
            from sklearn import cluster
            from sklearn import metrics

            # read embedding from file
            with open(Path.joinpath(embedding_save_path, f'{self.model_name}_vocab_embedding.txt')) as f:
              lines = f.readlines()
            embedding_data = []
            words_data = []
            for t in lines:
              w = t.split("\t")[0]
              v = [float(e) for e in t.split("\t")[1].split(" ")]
              words_data.append(w)
              embedding_data.append(v)
            # using kmean to get clusters of words
            kmeans = cluster.KMeans(n_clusters=10)
            kmeans.fit(embedding_data)
            labels = kmeans.labels_
            centroids = kmeans.cluster_centers_
            #print("Cluster id labels for inputted data")
            #print(labels)
            #print("Centroids data")
            #print(centroids)
            # dimension reduction with umap
            reducer = umap.UMAP(random_state=42,n_components=n_components)
            embedding = reducer.fit_transform(embedding_data)
            # show samples after dim-reduction in dataframe
            if n_components == 3:
                  wb = pd.DataFrame(embedding, columns=['x', 'y', 'z'])
            else:
                  wb = pd.DataFrame(embedding, columns=['x', 'y'])
            wb['word'] = words_data
            wb['cluster'] = ['cluster ' + str(c) for c in labels]
            # visualization with plotply
            if n_components==3:
                  fig = px.scatter_3d(wb, 
                                    text = wb['word'],
                                    x='x', y='y', z='z',
                                    color = wb['cluster'],
                                    title ="word-embedding-samples")
            else:
                  # n_components = 2
                  if text:
                    fig = px.scatter(wb, text = wb['word'], x='x', y='y', color=wb['cluster'], title='word embedding samples')
                  else:
                    fig = px.scatter(wb, x='x', y='y', color=wb['cluster'], title='word embedding samples')
                    
            fig.write_image(Path.joinpath(fig_path, f'embedding_space_dim_{n_components}.png'))
            fig.write_html(Path.joinpath(fig_path, f'embedding_space_dim_{n_components}.html'))
            fig.show()
            return True

  
class BertEmbedding:
    def __init__(self, saved_embeddings_text_file):
          self.file_path = saved_embeddings_text_file
          self.bert_vocab = None
          self.bert_embeddings = None
          self.bert_norms = None
    def get_bert_embeddings(self, etm_vocab):
          # filtering words by etm_vocab
          words_in_vocab, vocab_embeddings = read_prefitted_embedding("bert", etm_vocab, self.file_path)
          self.bert_embeddings = np.array(vocab_embeddings)
          self.bert_vocab = words_in_vocab
          self.bert_norms = np.array()
          print("bert-embedding ready!")
          return True

    def find_similar_words(self, word, top_neighbors):
          word_idx_in_vocab = self.bert_vocab.index(word)
          considered_vector = self.bert_embeddings[word_idx_in_vocab]
          top_words = get_similar_vectors_to_given_vector(top_neighbors, self.bert_vocab, considered_vector, self.bert_embeddings)
          return top_words

In [ ]:
Path('prepared_data').mkdir(parents=True, exist_ok=True)
Path(f'prepared_data/min_df_{min_df}').mkdir(parents=True, exist_ok=True)

save_path = Path.joinpath(Path.cwd(), f'prepared_data/min_df_{min_df}')
figures_path = Path.joinpath(Path.cwd(), f'figures/min_df_{min_df}')
Path(figures_path).mkdir(parents=True, exist_ok=True)

wb_creator = WordEmbeddingCreator(model_name="skipgram", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=0, embedding_size= 300)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

print("neighbor words of some sample selected words")
for i in range(0,5):
      print(f'neighbor of word {vocab[i]}')
      print([r[0] for r in wb_creator.find_most_similar_words(n_neighbor=5, word=vocab[i])])
      print([r[1] for r in wb_creator.find_most_similar_words(n_neighbor=5, word=vocab[i])])
      print(100*"-")

# Vergleichen ähnliche Wörter von Word2Vec

In [14]:
v = vocab[2]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'dim of vector: {len(vec)}')
print(wb_creator.find_most_similar_words(n_neighbor=10, word=v))

word-embedding of the word-- chung: 
dim of vector: 300
[('yang', 0.9316569566726685), ('coos', 0.9261748790740967), ('eecs', 0.9193140268325806), ('elroy', 0.9053330421447754), ('msd', 0.8993813991546631), ('orst', 0.8984891176223755), ('cae', 0.8945136070251465), ('speedy', 0.8906283378601074), ('starbase', 0.8893750905990601), ('sdsu', 0.8859439492225647)]


In [15]:
wb_creator.find_similar_words_self_implemented(10, vocab, v)

paperwork
[0.99999976, 0.9316569, 0.9261748, 0.91931397, 0.905333, 0.89938134, 0.89848906, 0.8945134, 0.8906283, 0.8893751]
[0.9316569, 0.9261748, 0.91931397, 0.905333, 0.89938134, 0.89848906, 0.8945134, 0.8906283, 0.8893751, 0.8859439]
[6049, 2457, 2609, 2709, 5935, 8494, 7875, 4724, 8147, 7949]


{'yang': 0.9316569,
 'coos': 0.9261748,
 'eecs': 0.91931397,
 'elroy': 0.905333,
 'msd': 0.89938134,
 'orst': 0.89848906,
 'cae': 0.8945134,
 'speedy': 0.8906283,
 'starbase': 0.8893751,
 'sdsu': 0.8859439}

# Vergleich ähnliche Wörter zwischen Word2Vec und Bert